# Тематическе моделирование

Что надо сделать:
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [ ]:
# Рабочий вариант зависимостей для визуализации результатов работы LDA модели
#  на момент август 2023 года.
# !pip install pandas==1.5.3
# !pip install numpy==1.18
!pip install -q pyLDAvis

In [2]:
!pip install -q pymorphy2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
import json
import string
import pymorphy2
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import ldamodel, CoherenceModel
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('drive/MyDrive/positive.csv',
                       sep=';',
                       usecols=[3],
                       names=['text'])
negative = pd.read_csv('drive/MyDrive/negative.csv',
                       sep=';',
                       usecols=[3],
                       names=['text'])

df = pd.concat([positive, negative])

In [6]:
df.shape

(226834, 1)

In [7]:
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


# Preprocessing

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
nltk.download('stopwords')
words_regex = re.compile('\w+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Загрузим дополнительные стоп-слова
!wget https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.json

--2023-08-16 04:46:42--  https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13060 (13K) [text/plain]
Saving to: ‘stopwords-ru.json’

stopwords-ru.json   100%[===================>]  12.75K  --.-KB/s    in 0.008s  

2023-08-16 04:46:42 (1.58 MB/s) - ‘stopwords-ru.json’ saved [13060/13060]



In [11]:
f = open('stopwords-ru.json')
data = json.load(f)
f.close()

In [12]:
extended_words = list(data)
stopwords_list = stopwords.words('russian') + extended_words

In [13]:
def find_words(text, regex=words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer=morph, stopwords=stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [14]:
df.text.iloc[1]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [15]:
print(preprocess(df.text.iloc[1]))

['всё', 'таки', 'немного', 'похожий', 'мальчик', 'всё', 'равно', 'хороший']


In [16]:
preprocessed_text = list(tqdm(map(preprocess, df['text']), total=len(df), desc='Progress'))

Progress:   0%|          | 0/226834 [00:00<?, ?it/s]

In [17]:
df['preprocessed_text'] = preprocessed_text
df.head(3)

,text,preprocessed_text
0,"@first_timee хоть я и школота, но поверь, у на...","[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...","[всё, таки, немного, похожий, мальчик, всё, ра..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[katiacheh, идиотка, испугаться]"


# LDA Modeling

In [18]:
!pip install -U -q gensim

Модель использует векторное представление документов, поэтому создадим словарь

In [19]:
dictionary = corpora.Dictionary(df['preprocessed_text'])

# игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря
dictionary.filter_extremes(no_below=10, no_above=0.9, keep_n=None)
dictionary.save('tweets.dict')

Векторизуем документы

In [20]:
corpus = [dictionary.doc2bow(text) for text in df['preprocessed_text']]
corpora.MmCorpus.serialize('tweets.model', corpus)

Обучение модели

In [21]:
%time

lda = ldamodel.LdaModel(corpus, id2word=dictionary,
                        num_topics=20,
                        chunksize=50,
                        update_every=1,
                        passes=2)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [22]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(10,
  '0.277*"пиздец" + 0.269*"нравиться" + 0.148*"прям" + 0.086*"свет" + 0.041*"мужчина" + 0.039*"контрольный" + 0.035*"рада" + 0.011*"заебись" + 0.004*"костюм" + 0.000*"сука"'),
 (9,
  '0.848*"самый" + 0.005*"победить" + 0.000*"жизнь" + 0.000*"ужасный" + 0.000*"обидный" + 0.000*"маленький" + 0.000*"экзамен" + 0.000*"старший" + 0.000*"читать" + 0.000*"вообще"'),
 (3,
  '0.000*"снегурочка" + 0.000*"фикбук" + 0.000*"обидеть" + 0.000*"побриться" + 0.000*"лавочка" + 0.000*"офигительный" + 0.000*"одинаково" + 0.000*"строй" + 0.000*"смелый" + 0.000*"mashushnik"'),
 (5,
  '0.264*"год" + 0.264*"сидеть" + 0.162*"новый" + 0.106*"папа" + 0.070*"взять" + 0.051*"жрать" + 0.026*"остальной" + 0.009*"читатель" + 0.000*"дом" + 0.000*"делать"'),
 (14,
  '0.000*"снегурочка" + 0.000*"фикбук" + 0.000*"обидеть" + 0.000*"побриться" + 0.000*"лавочка" + 0.000*"офигительный" + 0.000*"одинаково" + 0.000*"строй" + 0.000*"смелый" + 0.000*"mashushnik"'),
 (15,
  '0.397*"день" + 0.249*"блин" + 0.087*"посмотреть" 

Полученные темы спроецируем на плоскости

In [23]:
%time

pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [27]:
print('Perplexity: ', np.exp(lda.log_perplexity(corpus)))

Perplexity:  3.721755974075798e-12


In [28]:
coherence_model_lda = CoherenceModel(model=lda, texts=df['preprocessed_text'],
                                     dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Average coherence: ', coherence_lda)

Average coherence:  0.321211486570233
